In [ ]:
# Conexao com banco de dados
import psycopg2

try:
    conn = psycopg2.connect("dbname = 'DAGaula1' port = '5432'  user= 'user' password = 'user' host='localhost'")
    cursor = conn.cursor()
except:
    print 'Erro na conexao com o banco de dados'

# Interacao com o usuario = bairro
bairro = raw_input ("Entre com o nome do bairro: ")

# Calculo da area do bairro selecionado
cursor.execute("""SELECT ST_Area(divisa_de_bairros.geom) FROM divisa_de_bairros WHERE divisa_de_bairros.nome = '%s'""" % bairro)
rows = cursor.fetchall()
print "\nArea do bairro %s em metros:\n" % bairro
for row in rows:
    print "   ", (row[0])

# Consulta de selecao do bairro
cursor.execute("""CREATE OR REPLACE VIEW bairro_selection AS SELECT divisa_de_bairros.* FROM divisa_de_bairros WHERE divisa_de_bairros.nome = '%s'""" % bairro)

# Consulta dos bairros adjacentes
cursor.execute("""SELECT divisa_de_bairros.nome FROM divisa_de_bairros, bairro_selection WHERE ST_Touches(divisa_de_bairros.geom, bairro_selection.geom)""")
rows = cursor.fetchall()
print "\nBairros adjacentes ao %s\n" % bairro
for row in rows:
    print "   ", (row[0])
    
# Interacao com o usuario = distacia
dist = raw_input ("Entre com uma distancia: ")

# Consulta de selecao das escolas conforme a distancia
cursor.execute("""SELECT escola_municipal.nome FROM escola_municipal, bairro_selection WHERE ST_DWithin(escola_municipal.geom, bairro_selection.geom, %s)""" % dist)
rows = cursor.fetchall()
print "\nEscolas distantes de %s metros do bairro %s\n" % (dist, bairro)
for row in rows:
    print "   ", (row[0])

# Interacao com o usuario = coordenadas
E = raw_input ("Entre com a coordenada E (em metros): ")
N = raw_input ("Entre com a coordenada N (em metros): ")

# Consulta do bairro no qual caem as coordenadas
cursor.execute("""SELECT divisa_de_bairros.nome FROM divisa_de_bairros WHERE ST_Contains(divisa_de_bairros.geom, ST_SetSRID(ST_MakePoint(%s,%s),31982))""" % (E, N))
rows = cursor.fetchall()
print "\nAs coordenadas (%s, %s) caem no bairro:\n" % (E, N)
for row in rows:
    print "   ", (row[0])


conn.commit()
conn.commit()
cursor.close()
conn.close()


In [1]:
# Conexao com banco de dados
import psycopg2
import folium

m = folium.Map(location = [-25.45,-49.26], zoom_start = 12)

try:
    conn = psycopg2.connect("dbname = 'DAGaula1' port = '5432'  user= 'user' password = 'user' host='localhost'")
    cursor = conn.cursor()
except:
    print 'Erro na conexao com o banco de dados'

# Interacao com o usuario = bairro
bairro = raw_input ("Entre com o nome do bairro: ")

# Calculo da area do bairro selecionado
cursor.execute("""SELECT ST_Area(divisa_de_bairros.geom) FROM divisa_de_bairros WHERE divisa_de_bairros.nome = '%s'""" % bairro)
rows = cursor.fetchall()
print "\nArea do bairro %s em metros:\n" % bairro
for row in rows:
    print "   ", (row[0])

# Consulta de selecao do bairro
cursor.execute("""CREATE OR REPLACE VIEW bairro_selection AS SELECT divisa_de_bairros.* FROM divisa_de_bairros WHERE divisa_de_bairros.nome = '%s'""" % bairro)

# Consulta dos bairros adjacentes
cursor.execute("""CREATE OR REPLACE VIEW bairros_adja AS SELECT divisa_de_bairros.* FROM divisa_de_bairros, bairro_selection WHERE ST_Touches(divisa_de_bairros.geom, bairro_selection.geom)""")

# Interacao com o usuario = distacia
dist = raw_input ("Entre com uma distancia: ")

# Consulta de selecao das escolas conforme a distancia
cursor.execute("""CREATE OR REPLACE VIEW escola_selection AS SELECT escola_municipal.* FROM escola_municipal, bairro_selection WHERE ST_DWithin(escola_municipal.geom, bairro_selection.geom, %s)""" % dist)

# Interacao com o usuario = coordenadas
E = raw_input ("Entre com a coordenada E (em metros): ")
N = raw_input ("Entre com a coordenada N (em metros): ")

# Consulta do bairro no qual caem as coordenadas
cursor.execute("""CREATE OR REPLACE VIEW bairro_coord AS SELECT divisa_de_bairros.* FROM divisa_de_bairros WHERE ST_Contains(divisa_de_bairros.geom, ST_SetSRID(ST_MakePoint(%s,%s),31982))""" % (E, N))

# Interacao com o usuario = coordenadas
print 'A seguir digite as coordenadas de 4 pontos (latitude, longitude)'
lat1 = raw_input ("Entre com a latitude do ponto 1: ")
long1 = raw_input ("Entre com a longitude do ponto 1: ")

lat2 = raw_input ("Entre com a latitude do ponto 2: ")
long2 = raw_input ("Entre com a longitude do ponto 2: ")

lat3 = raw_input ("Entre com a latitude do ponto 3: ")
long3 = raw_input ("Entre com a longitude do ponto 3: ")

lat4 = raw_input ("Entre com a latitude do ponto 4: ")
long4 = raw_input ("Entre com a longitude do ponto 4: ")

# Deletar a tabela anterior. Criar uma nova tabela dos poligonos. Formar um poligono com as coordenadas.
# Na hora de fazer a interseccao deve transformar
cursor.execute("""DELETE FROM poligonos""")
cursor.execute("""INSERT INTO poligonos (geom) VALUES (ST_GeometryFromText('POLYGON ((%s %s, %s %s, %s %s, %s %s, %s %s))', 4326))""" % (long1, lat1, long2, lat2, long3, lat3, long4, lat4, long1, lat1))

conn.commit()
cursor.close()
conn.close()

# Camadas do Mapa
# Bairro selecionado
folium.raster_layers.WmsTileLayer(
    name = "Selecionado",
    url = "http://localhost:8082/geoserver/dag/wms",
    format = "image/png",
    layers = 'dag:bairro_selection',
    transparent = 'true'
).add_to(m)

# Bairro adjacente

folium.raster_layers.WmsTileLayer(
    name = "Adjacente",
    url = "http://localhost:8082/geoserver/dag/wms",
    format = "image/png",
    layers = 'dag:bairros_adja',
    transparent = 'true'
).add_to(m)

# Escolas selecionadas

folium.raster_layers.WmsTileLayer(
    name = "Escolas",
    url = "http://localhost:8082/geoserver/dag/wms",
    format = "image/png",
    layers = 'dag:escola_selection',
    transparent = 'true'
).add_to(m)

# Bairro que caem as coordenadas

folium.raster_layers.WmsTileLayer(
    name = "Bairro_coord",
    url = "http://localhost:8082/geoserver/dag/wms",
    format = "image/png",
    layers = 'dag:bairro_coord',
    transparent = 'true'
).add_to(m)

# Interseccao do poligono com os bairros

folium.raster_layers.WmsTileLayer(
    name = "pol_intersec",
    url = "http://localhost:8082/geoserver/dag/wms",
    format = "image/png",
    layers = 'dag:pol_intersec',
    transparent = 'true'
).add_to(m)

m

Entre com o nome do bairro: UBERABA

Area do bairro UBERABA em metros:

    14208267.059
Entre com uma distancia: 1000
Entre com a coordenada E (em metros): 679782.695
Entre com a coordenada N (em metros): 7188368.913
A seguir digite as coordenadas de 4 pontos (latitude, longitude)
Entre com a latitude do ponto 1: -25.47964
Entre com a longitude do ponto 1: -49.31732
Entre com a latitude do ponto 2: -25.47129
Entre com a longitude do ponto 2: -49.25414
Entre com a latitude do ponto 3: -25.53047
Entre com a longitude do ponto 3: -49.25088
Entre com a latitude do ponto 4: -25.53192
Entre com a longitude do ponto 4: -49.31006


In [1]:
# Conexao com banco de dados
import psycopg2

try:
    conn = psycopg2.connect("dbname = 'DAGaula1' port = '5432'  user= 'user' password = 'user' host='localhost'")
    cursor = conn.cursor()
except:
    print 'Erro na conexao com o banco de dados'

# Interacao com o usuario = coordenadas
print 'A seguir digite as coordenadas de 4 pontos (latitude, longitude)'
lat1 = raw_input ("Entre com a latitude do ponto 1: ")
long1 = raw_input ("Entre com a longitude do ponto 1: ")

lat2 = raw_input ("Entre com a latitude do ponto 2: ")
long2 = raw_input ("Entre com a longitude do ponto 2: ")

lat3 = raw_input ("Entre com a latitude do ponto 3: ")
long3 = raw_input ("Entre com a longitude do ponto 3: ")

lat4 = raw_input ("Entre com a latitude do ponto 4: ")
long4 = raw_input ("Entre com a longitude do ponto 4: ")

# Deletar a tabela anterior. Criar uma nova tabela dos poligonos. Formar um poligono com as coordenadas.
# Na hora de fazer a interseccao deve transformar
cursor.execute("""DELETE FROM poligonos""")
cursor.execute("""INSERT INTO poligonos (geom) VALUES (ST_GeometryFromText('POLYGON ((%s %s, %s %s, %s %s, %s %s, %s %s))', 4326))""" % (long1, lat1, long2, lat2, long3, lat3, long4, lat4, long1, lat1))

conn.commit()
cursor.close()
conn.close()

A seguir digite as coordenadas de 4 pontos (latitude, longitude)
Entre com a latitude do ponto 1: -25.47964
Entre com a longitude do ponto 1: -49.31732
Entre com a latitude do ponto 2: -25.47129
Entre com a longitude do ponto 2: -49.25414
Entre com a latitude do ponto 3: -25.53047
Entre com a longitude do ponto 3: -49.25088
Entre com a latitude do ponto 4: -25.53192
Entre com a longitude do ponto 4: -49.31006
